# Setup

In [ ]:
!pip list

## Installations

In [8]:
%pip install -r "requirements.txt"

Note: you may need to restart the kernel to use updated packages.


## Imports

In [11]:
import pandas as pd
import gffpandas.gffpandas as gffpd
from py2neo import Graph, Node, Relationship, cypher
from bisect import bisect_left as bisect

In [12]:
from typing import Sequence, Callable
from random import Random, randrange
import Levenshtein as lev
from multiprocessing.pool import ThreadPool as Pool

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModel

/home/daftylooper/Desktop/Capstone/code/finetune-chatgpt-go/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import transformers
transformers.__version__

'4.29.2'

In [16]:
def thread_starmap(arg_list, *, threads, progress_step=255):
  def progress(f):
    def inner(i, args):
      if not i&progress_step: print('Invocation', i)
      return f(*args)

  def inner(fn):
    with Pool(threads) as p:
      print('Running map...')
      p.starmap(progress(fn), enumerate(arg_list))
      print('Map complete. Destroying pool')
    print('Pool complete.')
    return fn
  return inner

## Connection

In [20]:
import os
from dotenv import load_dotenv
load_dotenv()

graph = Graph("neo4j+s://"+os.getenv("neo4j_connection_url"), auth=(os.getenv("neo4j_user"), os.getenv("neo4j_password")))

# Check connection
graph.run("RETURN 1")

1
1


# Main Graph

In [ ]:
def add_triple_to_graph(i, triple):

  if not i&255: print('Triple number', i)
      
  h, r, t = triple
  # Create or match the nodes, 'name' is a property
  head_node = Node("Entity", name=h)
  tail_node = Node("Entity", name=t)

  # Merge ensures nodes are only created if they don't already exist
  graph.merge(head_node, "Entity", "name")
  graph.merge(tail_node, "Entity", "name")

  # Create the relationship (head_node)-[relation]->(tail_node)    
  relationship = Relationship(head_node, r, tail_node)

  # Merge the relationship into the graph
  graph.merge(relationship)

## Create Sequence and Identity nodes + relations

In [ ]:
import csv
exon_data = []
path = "./data/exons/exon_sequences.csv"
hgnc = set()
labels = ["gene_id","transcript_id","hgnc_id","gene_type","gene_name","exon_id","protein_id","sequence"]
count = 0
with open(path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        # hgnc.add(tuple(row))
        exon_data.append(dict(zip(labels, row)))

# print(f"number of unique hgnc ids {len(hgnc)}")
exon_data = exon_data[1:]
print(f"pulled {len(exon_data)} sequences!")
exon_data = [kv for kv in exon_data if len(kv)==len(labels)]
print(f"new length {len(exon_data)}")

SyntaxError: invalid syntax (418375083.py, line 3)

In [ ]:
# maybe make a decorator for the thread pool thingi?

# TODO: ADD GO ref ids to Identity nodes later
seq_node_id_node_query = '''
MERGE (s:Sequence {sequence: $sequence, exon_id: $exon_id, protein_id: $protein_id})-[:MAPS_TO]->(i:Identity {gene_id: $gene_id, hgnc_id: $hgnc_id, transcript_id: $transcript_id});
'''
def make_seq_node_with_id_node(i, d):
    # arr is of type - [ensembl_id, gene_type, gene_name, hgnc_id, seq]
    if not i&127: print(f"Processed {i} nodes!")
    graph.run(seq_node_id_node_query, **d)
    

In [ ]:
sequence_identity_node_query = '''
MERGE (s:Sequence {sequence: $sequence, exon_id: $exon_id, protein_id: $protein_id})
MERGE (t:Transcript {transcript_id: $transcript_id})
MERGE (g:Gene {gene_id: $gene_id, hgnc_id: $hgnc_id})
MERGE (s)-[:MAPS_TO]->(t)
MERGE (t)-[:PART_OF]->(g);
'''
# identity_node_query = '''
# MERGE (i:Identity {gene_id: $gene_id, hgnc_id: $hgnc_id, transcript_id: $transcript_id})
# MATCH (e:Sequence {exon_id: $exon_id}), (i:Identity {gene_id: $gene_id})
# MERGE (e)-[:MAPS_TO]->(i);
# '''
# sequence_identity_relation_query = '''
# MATCH (e:Sequence {exon_id: $exon_id}), (i:Identity {gene_id: $gene_id})
# MERGE (e)-[:MAPS_TO]->(i);
# '''
count = 0
total = len(exon_data)
def make_seq_node_with_id_node(i, d):
    global count, total
    # arr is of type - [ensembl_id, gene_type, gene_name, hgnc_id, seq]
    if not count&127: print(f"Processed {count}/{total} nodes!")
    graph.run(sequence_identity_node_query, **d)
    count+=1

In [ ]:
print('Creating pool')
with Pool(100) as p:
  print('Running map...')
  p.starmap(make_seq_node_with_id_node, enumerate(exon_data))
  print('Queries complete. Destroying pool')
print('Pool complete.')

## Map Ensembl ID to Go

In [ ]:
import csv

labels = ["gene_id", "go"]
path = "./data/go/ensembl_go_protein_coding.csv"
go_data = []
go = set()
with open(path, "r") as f:
    reader = csv.reader(f)
    for row in reader:
        go_data.append(dict(zip(labels, row)))
        # go.add(row[1])

go_data = go_data[1:]
print(f"pulled {len(go_data)} records!")
# print(f"unique go ids: {len(go)}")

In [ ]:
gene_go_mapper_query = '''
MATCH (s: Gene {gene_id: $gene_id})
MERGE (go: Go {go_id: $go})
MERGE (s)-[:HAS_GO]->(go);
'''
count = 0
total = len(go_data)
def map_gene_to_go(i, d):
    global count, total
    if not count&127: print(f"Processed {count}/{total} nodes!")
    graph.run(gene_go_mapper_query, **d)
    count+=1

In [ ]:
print('Creating pool')
with Pool(100) as p:
  print('Running map...')
  p.starmap(map_gene_to_go, enumerate(go_data))
  print('Queries complete. Destroying pool')
print('Pool complete.')

In [ ]:
import csv

go_triples = []
path = "./data/go/go_triples_protein_coding.csv"
labels = ["go_id", "relation_type", "property"]

with open(path, "r") as f:
    reader = csv.reader(f)
    for row in reader:
        go_triples.append(dict(zip(labels, row)))

go_triples = go_triples[1:]
print(go_triples[0])
print(go_triples[3])
print(f"pulled {len(go_triples)} go triples!")

In [ ]:
count = 0
total = len(go_triples)
add_go_triple_query = '''
MATCH (go:Go {go_id: $go_id})
MERGE (gop:GoProperty {property: $property})
'''
relation = "MERGE (go)-[:{relation_type}]->(gop)"

def add_go_triple_to_graph(i, triple):
    global count, total
    if not count&127 and count!=0: print(f"processed {count}/{total} triples!")
    r = relation.format(relation_type=triple["relation_type"])
    q = add_go_triple_query+r
    graph.run(q, **triple)
    count+=1

In [ ]:
print('Creating pool')
with Pool(100) as p:
  print('Running map...')
  p.starmap(add_go_triple_to_graph, enumerate(go_triples))
  print('Queries complete. Destroying pool')
print('Pool complete.')

# Search Tree

## Retrieving the Genes and their Node ids

In [8]:
id_map = graph.run('MATCH (g:Sequence) RETURN {g: g, id: elementId(g)}')

In [9]:
id_map = id_map.data()

In [10]:
len(id_map)
id_map[:10]

[{'{g: g, id: elementId(g)}': {'id': '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:0',
   'g': Node('Sequence', exon_id='ENSE00001943628', level=12, limit=1.2786107063293457, protein_id='ENSP00000466958', sequence='TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG', thresh=1.2786107063293457)}},
 {'{g: g, id: elementId(g)}': {'id': '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:1',
   'g': Node('Sequence', exon_id='ENSE00003602238', level=11, limit=1.5374518632888794, protein_id='ENSP00000317992', sequence='CTGGTGTCCTGTCTGTCGGAGACGACGGTGTTGGCGGCCGTGCTGCGGCACATCAGCGTGCTGGTGCCCTGCTTCCTGACCTTCCCCAAGCAGTGCCGCATGCTGCTCAAG', thresh=1.465311884880066)}},
 {'{g: g, id: elementId(g)}': {'id': '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:2',
   'g': Node('Sequence', exon_id='ENSE00001688790', level=11, limit=1.5303996801376343, protein_id='ENSP00000368678', sequence='GCCCCACCTTCGCCCCGCTGCCGCCTGTGGCCCCCTTACACT

In [11]:
id_pairs: list[tuple[str, str]] = [(v['id'], v['g']['sequence']) for r in id_map for v in [r.values().__iter__().__next__()]]

print(len(id_pairs), 'pairs')

id_pairs[:5]

19995 pairs


[('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:0',
  'TACAGGCAGAAAACTACCTGGCACTGTGGGAGACTGAGTATAGGAGAAAGCAGGATGACGATGGGCCTGAATGGTGGTGTGTTTTCCTCCACACAATGTGGCTTTGTCCTCCAAGGTGTCTCACGTGATGGAAGCTGGCTGATCTGACTTGCTTG'),
 ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:1',
  'CTGGTGTCCTGTCTGTCGGAGACGACGGTGTTGGCGGCCGTGCTGCGGCACATCAGCGTGCTGGTGCCCTGCTTCCTGACCTTCCCCAAGCAGTGCCGCATGCTGCTCAAG'),
 ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:2',
  'GCCCCACCTTCGCCCCGCTGCCGCCTGTGGCCCCCTTACACTGTGCCCAGACGCCCTACGGCTGCTGCCAGGACAATATCACCGCAGCCCGGGGCGTGGGCCTGGCTGGCTGCCCCA'),
 ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:3',
  'ATGCTCTACCTGCTGTGCTCCTGGCCGGAGCTGCCCGTCCTGAGCGCCCTGGAGCTGCTAGACTTCAGCTTCCCCGATTGCCACGTAGGCTCCTTCGCCATCAAGTCGCTGCGGAAACTGAC'),
 ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:4',
  'ATCGGCTCCGTGGAGCGCTTCATCCACCACGTGAACGCGTGCATGCGGCAGCGGCAGGAGCGGCAGCGGCTGGCGGCCGTGGTGAGCCGCATCGACGCCTACGAGGTGGTGGAAAGCAGCAGCGACGAAGTGGACAAG')]

In [12]:
import os

os.environ["WANDB_DISABLED"] = "true"
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
# cuda_model = model.to("cuda")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

configuration_bert.py:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- bert_padding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


flash_attn_triton.py:   0%|          | 0.00/42.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- bert_layers.py
- bert_padding.py
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/468M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:446: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

## Distance function using DNABERT-2

In [18]:
def dnabert_embedding_mean(dna):
    # inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"].to('cuda')
    inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]
    hidden_states = model(inputs)[0] # [1, sequence_length, 768]
    # print(inputs.shape)
    # print(hidden_states.shape)

    # embedding with mean pooling
    embedding_mean = torch.mean(hidden_states[0], dim=0)
    # print(embedding_mean.shape) # expect to be 768
    # print(embedding_mean)
    
    # # embedding with max pooling
    # embedding_max = torch.max(hidden_states[0], dim=0)[0]
    # print(embedding_max.shape) # expect to be 768
    
    return embedding_mean.detach()

In [19]:
def dnabert_embedding_distance(a, b):

    # print(f"got -> \n{a}\n{b}")
    try:
        vd1 = memo[a]
    except KeyError:
        print("KeyError: a:", a)
        
    try:
        vd2 = memo[b]
    except KeyError:
        print("KeyError: b:", b)
    
    return float(torch.linalg.vector_norm(vd1-vd2))

## Calculate and memoize embeddings. Store for future sessions.

In [16]:
import numpy as np

def embedding_maker_worker(i, tup):
    id, seq = tup
    with torch.no_grad():
        if not i&127: print(f"{i}: {len(seq)}")
        arr.append(len(seq))
        memo[seq] = dnabert_embedding_mean(seq)

# print(f"first seven - {arr[:8]}")
# print(f"avg seq len {np.mean(arr)}")

In [ ]:
import os
import ast
import csv

path = "./data/working"
memo = {}
arr = []

if os.path.exists(path):
    print("Found existing dump of embeddings, loading them...")
    with open(path, "r") as f:
        reader = csv.reader(f)
        for row in reader:
            seq, embedding = row
            embedding = torch.tensor(ast.literal_eval(embedding))
            memo[seq] = embedding
else:
    print("Embeddings don't exist, creating...")
    with Pool(200) as p:
      print('Running map...')
      p.starmap(embedding_maker_worker, enumerate(id_pairs[::-1]))
      print('Queries complete. Destroying pool')

    print("Writing Rows...")
    with open(path, "w") as f:
        writer = csv.writer(f)
        for seq, embd in memo.items():
            writer.writerow([seq, embd.tolist()])

print("Done!")

Found existing dump of embeddings, loading them...
Done!


In [ ]:
# !rm "./data/working/dnabert2_embeddings.csv"

In [17]:
memo["CCCAGATCTCTTCAG"]
print(f"size of id_pairs: {len(id_pairs)}")
print(f"size of memo: {len(memo)}")

tensor([ 7.8781e-02,  8.3097e-02,  1.3763e-01, -6.9165e-02, -1.5011e-01,
         1.2471e-02, -2.3310e-01,  1.6233e-03,  1.3307e-01, -2.3262e-01,
        -1.7740e-01,  1.6837e-01, -7.9002e-02, -3.9307e-01,  2.8540e-02,
        -6.5927e-03, -7.7574e-03, -4.8451e-02, -9.7568e-02, -1.2833e-01,
         2.8347e-01, -2.8138e-02,  8.5315e-02, -2.5344e-01, -5.5409e-02,
         1.1990e-01, -1.5642e-01, -5.3364e-02, -5.2738e-02,  1.5500e-01,
        -7.5869e-02,  2.2803e-01,  9.0783e-02,  4.7768e-02,  1.1843e-02,
        -4.1791e-03, -2.2109e-01,  2.7762e-01, -5.1653e-02,  3.1720e-02,
        -4.0129e-01, -3.8747e-01, -4.0016e-02, -2.3743e-01,  1.0303e-01,
        -1.1825e-01,  3.3379e-02, -6.8411e-02, -7.8926e-02,  1.2446e-01,
        -2.1545e-01, -3.4480e-02, -2.7330e-02,  4.9683e-02,  3.0291e-01,
         1.9072e-01,  6.3323e-02, -2.9824e-02, -8.2428e-02, -1.8449e-01,
        -2.4261e-01,  3.1160e-02,  5.3444e-02,  3.6625e-01,  3.1646e-01,
         4.3492e-02,  1.3542e-02,  2.1366e-01,  1.3

In [19]:
print(dnabert_embedding_distance(
  "CCCAGATCTCTTCAG",
  "TTTTTATGCCTCATTCTGTGAAAATTGCTGTAGTCTCTTCCAGTTATGAAGAAG",
))

print(len(memo))

6.9764404296875
8472


## Construction

In [20]:
query = '''

  MATCH (r:Sequence WHERE elementId(r) = $root_id)

  MATCH (n:Sequence WHERE elementId(n) = $n_id)

  SET r.thresh = $thresh

  SET r.limit = $limit

  SET n.level = $level

  CREATE (r)-[:SEARCH_BRANCH {branch_type: $branch_type}]->(n)

'''

In [ ]:
def create_tree_instructions(graph, id_pairs: list[tuple[str, str]], distance: Callable[[Sequence, Sequence], float], r: Random, level = 0) -> tuple[str, str, list[dict]] | None:

  if not id_pairs: return None

  out = []



  l = len(id_pairs)

  root_idx = r.randrange(l)

  root_id, root = id_pairs.pop(root_idx)



  s = sorted(id_pairs, key=lambda g: distance(root, g[1]))

  split_pos = l//2

  if not s: limit = thresh = 0.0

  elif l//2 >= len(s): limit = thresh = distance(root, s[-1][1])

  else:
    print(s)

    distances = [distance(root, g[1]) for g in s]

    thresh = distances[l//2]

    limit = distances[-1]

    # if limit != thresh: print('EXPECTED', thresh, limit)

    split_pos = bisect(distances, thresh)

    # out.append({'id': root_id, 'limit': limit, 'thresh': thresh, 'far_subtree': (distances[split_pos], distances[-1])})



  near_result = create_tree_instructions(graph, s[:split_pos], distance, r, level=level+1)

  if near_result is not None:

    out.extend(near_result[2])

    out.append({

        "branch_type": "NEAR",

        "root_id": root_id,

        "n_id": near_result[0],

        "thresh": thresh,

        "level": level+1,

        "limit": limit,

    })



  far_result = create_tree_instructions(graph, s[split_pos:], distance, r, level=level+1)

  if far_result is not None:

    out.extend(far_result[2])

    out.append({

        "branch_type": "FAR",

        "root_id": root_id,

        "n_id": far_result[0],

        "thresh": thresh,

        "level": level+1,

        "limit": limit,

    })



  print(level, root_id, root[:64])



  return root_id, root, out

In [ ]:
def edit_distance(a: Sequence, b: Sequence) -> float:  # levenshtein distance

  ai = 0

  bi = 0

  d = 0.0

  while ai < len(a) and bi < len(b):

    if a[ai] == b[bi]:  # no edit

      ai += 1

      bi += 1

    elif ai > 0 and a[ai-1] == b[bi]:  # add to a / delete from b

      d += 1

      bi += 1

    elif bi > 0 and a[ai] == b[bi-1]:  # add to b / delete from a

      d += 1

      ai += 1

    else:  # replacement

      d += 1

      ai += 1

      bi += 1



  d += len(a)-ai

  d += len(b)-bi



  return d

In [ ]:
# seed = randrange(1<<32)

# seed = 0x509c2232

seed = 0xf67832cb

print(f'seed = 0x{seed:08x}')

r = Random(seed)
     
# root_id, root, kwarg_list = create_tree_instructions(graph, id_pairs.copy(), edit_distance, r)
root_id, root, kwarg_list = create_tree_instructions(graph, id_pairs.copy(), dnabert_embedding_distance, r)

root_id, root

In [ ]:
# Takes 1-2 seconds. *Blazingly Fast*

print('Creating pool')

with Pool(100) as p:

  print('Running map...')

  p.map(

    lambda kwargs: graph.run(query, **kwargs),

    kwarg_list,

  )

  print('Queries complete. Destroying pool')

print('Pool complete.')

## Verification

In [ ]:
def tree_testing(graph, id_pairs: list[tuple[str, str]], distance: Callable[[Sequence, Sequence], float], r: Random, level = 0) -> tuple[str | None, str | None, list[dict]]:

  if not id_pairs: return None, None, []

  out = []



  l = len(id_pairs)

  root_idx = r.randrange(l)

  root_id, root = id_pairs.pop(root_idx)



  s = sorted(id_pairs, key=lambda g: distance(root, g[1]))

  split_pos = l//2

  if not s:

    thresh = 0.0

    limit = 0.0

  elif l//2 >= len(s): limit = thresh = distance(root, s[-1])

  else:

    distances = [distance(root, g[1]) for g in s]

    thresh = distances[l//2]

    limit = distances[-1]

    if limit != thresh: print('EXPECTED', thresh, limit)

    split_pos = bisect(distances, thresh)

    out.append({'id': root_id, 'limit': limit, 'thresh': thresh, 'far_subtree': (distances[split_pos], distances[-1])})



  near_result = tree_testing(graph, s[:split_pos], distance, r, level=level+1)

  out.extend(near_result[2])

  far_result = tree_testing(graph, s[split_pos:], distance, r, level=level+1)

  out.extend(far_result[2])



  if level < 7: print(level, root_id, root[:64])



  return root_id, root, out



seed = 0xf67832cb

print(f'seed = 0x{seed:08x}')

r = Random(seed)

# root_id, root, lev_properties = tree_testing(graph, id_pairs.copy(), lev.distance, r)

root_id, root, lev_properties = tree_testing(graph, id_pairs.copy(), edit_distance, r)

root_id, root

In [ ]:
[prop for prop in lev_properties if len({*prop['far_subtree']}) == 2]

In [ ]:
len(lev_properties)

In [ ]:
root_id

In [ ]:
len(kwarg_list)

In [ ]:
# Check if update was successful

l = [(d['r']['thresh'], d['r']['limit']) for d in graph.run('MATCH  (r:Sequence)-[:SEARCH_BRANCH]->(n) RETURN r, n').data() if d['r']['thresh'] != d['r']['limit']]

print(len(l))

l

In [21]:
unique_kwargs = {(kwarg['n_id'], kwarg['root_id']) for kwarg in kwarg_list}

len(unique_kwargs)

NameError: name 'kwarg_list' is not defined

In [ ]:
for i, kwargs in enumerate(kwarg_list[:15]):

  print('''

    MATCH (r:Gene WHERE elementId(r) = {root_id!r})

    MATCH (n:Gene WHERE elementId(n) = {n_id!r})

    SET r.thresh = {thresh!r}

    CREATE (r)-[:SEARCH_BRANCH {{branch_type: {branch_type!r}}}]->(n)

    RETURN 1

  '''.format(**kwargs))

## Search

In [48]:
def search(root_id, root_name, tree_thresh, tree_limit, q, qthresh, paths=1, distance=dnabert_embedding_distance) -> tuple[list[tuple[float, str, str]], int]:

  # print("dist->")
  # print(q)
  # print(root_name)
  if not tree_thresh and not tree_limit:
      return ([], paths)
      
  d = distance(q, root_name)  

  if tree_limit <= qthresh-d:

    print('SUBTREE BATCH')

    results = graph.run(

      '''

      MATCH (r WHERE elementId(r) = $root_id)-[:SEARCH_BRANCH*1..]->(c)

      RETURN {child_id: elementId(c), child: c.sequence}

      ''',

      root_id=root_id,

    ).data()



    out = []

    for result in results:

      [values] = [*result.values()]

      out.append((distance(q, values['child']), values['child_id'], values['child']))

    return out, paths



  elif d <= qthresh:

    out = [(d, root_id, root_name)]

  else:

    out = []



  if tree_thresh is None: return out, paths



  result = graph.run(

    '''

    MATCH (r WHERE elementId(r) = $root_id)-[sb:SEARCH_BRANCH]->(c)

    RETURN {child_id: elementId(c), child: c.sequence, thresh: c.thresh, limit: c.limit, branch: sb.branch_type}

    ''',

    root_id=root_id,

  ).data()



  children: dict[str, tuple[str, str, str, str]] = {}

  for child in result:

    [values] = [*child.values()]

    children[values['branch']] = values['child_id'], values['child'], values['thresh'], values["limit"]



  if abs(d-tree_thresh) <= qthresh:

    paths += 1

    print(f'Paths: {paths}, Diff: {abs(d-tree_thresh)}')


  # print("near:", children["NEAR"])
  # print("far:", children["FAR"])
    
  if d-qthresh < tree_thresh and 'NEAR' in children:
    # print("NEAR: calling but not passing q:", q)
    terms, paths = search(*children['NEAR'], q, qthresh, paths)

    out.extend(terms)

  elif d+qthresh >= tree_thresh and 'FAR' in children:
    # print("FAR: calling but not passing q:", q)
    terms, paths = search(*children['FAR'], q, qthresh, paths)

    out.extend(terms)



  return (out, paths)

In [ ]:
q_result, paths = search(root_id, root, root_thresh, root_limit, root, 30000)

print(len(q_result), 'similar genes')

sorted(q_result)

## Searching on exon graph

In [26]:
root_thresh = 1.7634133100509644  # queried from the neo4j console for ({level: 0})
root_limit = 6.264886856079102  # queried from the neo4j console for ({level: 0})
root_id, root = ('4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:1956', 'GCCCCACGTGTGTGCTGAGCAGGAGCTGACCCTGGTGGGCCGCCGCCAGCCGTGCGTGCAGGCCTTAAGCCACACGGTGCCGGTGTGGAAGGCCGGCTGTGGGTGGCAGGCGTGGTGCGTGGGTCATGAGCGGAG')

In [49]:
q_result, paths = search(root_id, root, root_thresh, root_limit, "CCCAGATCTCTTCAG", 1)
print(len(q_result), 'similar genes')
print("results!\n-----------")
print(q_result)
print(paths)

Paths: 2, Diff: 0.5606060028076172
Paths: 3, Diff: 0.6051273345947266
Paths: 4, Diff: 0.0337529182434082
1 similar genes
results!
-----------
['GTCGTCTCCCGGCAGTGCAGCTGCCGCTACCGCCGCCCTCTGCCCGCCGGCCCGTCTGTCTACCCCCAGCATGAGCGGCCTGCGCGTCTACAGCACGTCGGTCACCGGCTCCCGCGAA']
4


In [58]:
def similarity_search(q):
    memo[q] = dnabert_embedding_mean(q)
    q_result, paths = search(root_id, root, root_thresh, root_limit, q, 1)
    return q_result

In [22]:
q = "CCCAGATCTCTTCAG"
# print(memo[q])
similarity_search("CCCAGATCTCTTCAG")

NameError: name 'search' is not defined

In [62]:
print(similarity_search(root))

seqs = ["GGGCCCCCTCATAAATGTGCCTTAATTTTCGCAGATAACAGTGGAATAGACATCATTTTGGGAGTCTTCCCCTTTGTCAGGGAGCTACTCCTTAGAGGGACAGAG",
        "AGCCTGAAGCCGCTGCTGGAGAAGCGCCGGCGCGCGCGCATCAACCAGAGCCTGAGCCAGCTTAAGGGGCTCATCCTGCCGCTGCTGGGCCGGGAG",
        "GTCTGTGATGAGGAATGGCACCACTACGTCCTCAATGTAGAATTCCCGAGTGTGACTCTCTATGTGGATGGCACGTCCCACGAGCCCTTCTCTGTGACTGAGGATTACCCGCTCCATCCATCCAAGATAGAAACTCAGCTCGTGGTGGGGGCTTGCTGGCAAG",
        "GCCCAGTTCAATCTGCTGAGCAGCACCATGGACCAGATGAGCAGCCGCGCGGCCTCGGCCAGCCCCTACACCCCAGAGCACGCCGCCAGCGTGCCCACCCACTCGCCCTACGCACAACCCAGCTCCACCTTCGACACCATGTCGCCGGCGCCTGTCATCCCCTCCAACACCGACTACCCCGGACCCCACCACTTTGAGGTCACTTTCCAGCAGTCCAGCACGGCCAAGTCAGCCACCTGGACG",
        "GCTCGCGGGACCCCTGCTCCAACGTGACCTGCAGCTTCGGCAGCACCTGTGCGCGCTCGGCCGACGGGCTGACGGCCTCGTGCCTGTGCCCCGCGACCTGCCGTGGCGCCCCCGAGGGGACCGTCTGCGGCAGCGACGGCGCCGACTACCCCGGCGAGTGCCAGCTCCTGCGCCGCGCCTGCGCCCGCCAGGAGAATGTCTTCAAGAAGTTCGACGGCCCTTGTG"]

out = []
for seq in seqs:
    out.append(similarity_search(seq))

print("answer->")
print(*out)

Paths: 2, Diff: 0.08228754997253418
Paths: 3, Diff: 0.07570505142211914
Paths: 4, Diff: 0.3529481887817383
Paths: 5, Diff: 0.46616554260253906
Paths: 6, Diff: 0.43332719802856445
Paths: 7, Diff: 0.3127783536911011
Paths: 8, Diff: 0.34830915927886963
Paths: 9, Diff: 0.33372581005096436
Paths: 10, Diff: 0.4161565899848938
Paths: 11, Diff: 0.5772742629051208
Paths: 12, Diff: 0.5513879060745239
Paths: 13, Diff: 0.6479724645614624
[(0.0, '4:c81c1c99-e897-44b9-b8cc-e4b6e264a420:1956', 'GCCCCACGTGTGTGCTGAGCAGGAGCTGACCCTGGTGGGCCGCCGCCAGCCGTGCGTGCAGGCCTTAAGCCACACGGTGCCGGTGTGGAAGGCCGGCTGTGGGTGGCAGGCGTGGTGCGTGGGTCATGAGCGGAG'), 'ACCCCAACACTCACATCCTCTACGCCTGTGACTCCTGTGGGGACAAGTTTCTGGATGCCAACAGCCTGGCTCAGCATGTGCGAATCCACACAGCCCAGGCACTGGTCATGTTCCAGACAGACGCGGACTTCTATCAGCAGTATGGGCCAGGTGGCACGTGGCCTGCCGGGCAGGTGCTGCAGGCTGGGGAGCTGGTCTTCCGCCCTCGCGACGGGGCTGAGGGCCAGCCCGCACTGGCAGAGACCTCCCCTACAGCTCCTGAATGTCCCCCGCCTGCCGAGTGAGCTGGCGGCCCTTCTGACTGTTTATTTAAGGATGGATGGCACCCTGGAACCGGGAAGGGTGGCCTGTTCCCTAGAGAGAATAAATTGGATT

# **Flask Server**